In [1]:
import os
import json
import time
import warnings
from tqdm.auto import tqdm
from contextlib import redirect_stdout, redirect_stderr

In [2]:
articles_dir = 'articles'

In [3]:
papers_paths_pdf = []
papers_paths_json = []
for journal in tqdm(os.listdir(articles_dir)):
    # check if journal name contains arabic letter, if not: skip
    arabic_letters = ['ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ي']
    if len(set(journal).intersection(set(arabic_letters))) == 0:
        continue
    for volume in os.listdir(f"{articles_dir}/{journal}"):
        volume_date = volume.split('-')[-1]
        if 2010 <= int(volume_date) <= 2022: # consider this range
            for issue in os.listdir(f"{articles_dir}/{journal}/{volume}"):
                for paper in os.listdir(f"{articles_dir}/{journal}/{volume}/{issue}"):
                    paper_name = '.'.join(paper.split('.')[:-1])
                    paper_path_pdf = f"{articles_dir}/{journal}/{volume}/{issue}/{paper_name}.pdf"
                    paper_path_json = f"{articles_dir}/{journal}/{volume}/{issue}/{paper_name}.json"
                    if paper_path_pdf not in papers_paths_pdf:
                        papers_paths_pdf.append(paper_path_pdf)
                    if paper_path_json not in papers_paths_json:
                        if os.path.exists(paper_path_json):
                            papers_paths_json.append(paper_path_json)

  0%|          | 0/368 [00:00<?, ?it/s]

In [5]:
len(papers_paths_pdf), len(papers_paths_json)

(133912, 133912)

# split abstracts by language

abstract comes as one element written with multiple languages.

In [6]:
import re
from langdetect import detect, DetectorFactory, lang_detect_exception

# Set seed for consistent results
DetectorFactory.seed = 0

def clean_text(text):
    """
    Cleans text by removing extra spaces and normalizing punctuation.
    """
    # Normalize spaces
    text = re.sub(r'\s+', ' ', text)
    # Ensure proper spacing after periods
    text = re.sub(r'\.(?=[A-Za-z])', '. ', text)
    return text.strip()

def get_sentences(text):
    """
    Splits text into sentences more accurately, handling both English and Arabic punctuation.
    """
    # Handle both Arabic and English sentence endings
    text = clean_text(text)
    # Split on period followed by space and capital letter, or Arabic sentence endings
    sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])|[.!?؟]\s+|الملخص:|الملخص :|.Abstract|Abstract :|Abstract:|Abstract|.Résumé|Résumé:|Résumé :|Résumé', text)
    return [s.strip() + '.' for s in sentences if s.strip()]

def detect_language_with_confidence(text):
    """
    Detects language with additional confidence checks.
    """
    if not text or not text.strip():
        return None
    
    try:
        # Count character types for additional verification
        arabic_chars = sum(1 for c in text if '\u0600' <= c <= '\u06FF')
        latin_chars = sum(1 for c in text if ('a' <= c.lower() <= 'z'))
        total_chars = len(text.replace(' ', ''))
        
        # If text is clearly dominated by one script, use that
        if arabic_chars / total_chars > 0.75:
            return 'arabic'
        elif latin_chars / total_chars > 0.75:
            detected = detect(text)
            return 'english' if detected == 'en' else ('french' if detected == 'fr' else None)
            
        # For mixed text, use langdetect
        detected = detect(text)
        return {
            'en': 'english',
            'fr': 'french',
            'ar': 'arabic'
        }.get(detected)
    except lang_detect_exception.LangDetectException:
        return None

def separate_abstracts(text):
    """
    Separates a multilingual text into different languages with improved boundary detection.
    """
    abstracts = {
        'english': [],
        'arabic': [],
        'french': []
    }
    
    # Clean and normalize the text
    text = clean_text(text)
    
    # Get sentences with improved splitting
    sentences = get_sentences(text)
    
    # Process each sentence
    for sentence in sentences:
        # Skip empty or very short sentences
        if not sentence or len(sentence.strip()) < 10:
            continue
            
        lang = detect_language_with_confidence(sentence)
        if lang:
            abstracts[lang].append(sentence)
    
    # Join the collected segments for each language
    result = {
        'english': ' '.join(abstracts['english']).strip('-').strip().strip(':').strip('-').strip().strip('summary').strip(':').strip() if abstracts['english'] else None,
        'arabic': ' '.join(abstracts['arabic']).strip('-').strip().strip(':').strip('-').strip().strip('ملخص').strip('ملخّص').strip('الملخص').strip('الملخّص').strip(':').strip() if abstracts['arabic'] else None,
        'french': ' '.join(abstracts['french']).strip('-').strip().strip(':').strip('-').strip() if abstracts['french'] else None
    }
    # detect the case of Abstract:blablabal
    for key in result:
        if result[key] and ':' in result[key].split()[0]:
            result[key] = ''.join(result[key].split(':')[1:]).strip()
            result[key] = ''.join(c for c in result[key] if c.isprintable())
            
    
    return result

In [7]:
# Example 1: Text with all three languages
text1 = """
English: This is an English abstract about science.
Arabic: هذا ملخص عربي عن العلوم
French: Ceci est un résumé français sur la science.
"""

text1 = """
ملخص: يتمثل الهدف الرئيسي من هذه الدراسة في توضيح الدور الذي يلعبه الاشهاد على مطابقة المنتوجات، أو حصول المؤسسة على شهادة المطابقة لمنتجاتها في ابراز مكانتها الاقتصادية على المستويين الوطني و الدولي، وبالتالي النهوض بالاقتصاد الوطني، خاصة ونحن على علم بأن الجزائر خاضت غمار الولوج إلى الأسواق العالمية من خلال فتحها باب الاستثمار، وأيضا الاصلاحات التي اتخذنها في أواخر الثمانينات وبداية التسعينات من خلال تبني نظام اقتصاد السوق، وسعيها للانضمام إلى منظمة التجارة العالمية.من خلال هذه الدراسة نحاول الاجابة على الاشكالية التي يمكن طرحها في هذا المجال المتمثلة في كيفية تعامل المشرع الجزائري مع نظام الاشهاد على المطابقة، بناء على أن المشرع هو الذي يضع الاطار القانوني لأي سياسة تنتهجها الدولة في جميع الميادين بما فيها الميدان الاقتصادي، وقد قمنا بدراسة هيئات تقييم المطابقة أولا، و إجراءات الاشهاد على المطابقة ثانيا. وخلصنا إلى نتيجة عامة هي لامبالاة المؤسسات الجزائرية بنظام الجودة ولا الاشهاد عليها خصوصا الصغيرة والمتوسطة منها.Abstract: The main objective of this study is to clarify the role played by the certification of conformity of products, or the institution obtaining a certificate of conformity for its products, in highlighting its economic position at national and international levels, and thus promoting the national economy, in particular because we know that Algeria entered the world market by opening the door to investments, as well as to the reforms it undertook in the late 1980s and early 1990s by adopting the market economy system and seeking membership in the World Trade Organization. Through this study, we try to respond to the problem that may arise in this field represented in the way in which the Algerian legislator treats the system of certification of conformity, starting from the fact that the legislator is the one who fixes the legal framework of any policy. pursued by the State in all fields, including the economic field, and we first studied conformity assessment bodies, then conformity certification procedures. We arrived at a general conclusion which is the indifference of Algerian institutions to the quality system and indifference to it, especially the small and medium ones.Résumé : L'objectif principal de cette étude est de clarifier le rôle que joue la certification de conformité des produits, ou l'institution obtenant un certificat de conformité pour ses produits, dans la mise en évidence de sa position économique aux niveaux national et international, et ainsi la promotion de l'économie nationale. , d'autant plus que nous savons que l'Algérie est entrée au milieu de l'accès aux marchés mondiaux en ouvrant la porte à l'investissement, ainsi que les réformes qu'elle a entreprises à la fin des années 80 et au début des années 90 en adoptant le système d'économie de marché, et leur quête pour rejoindre le L'organisation de commerce mondial. A travers cette étude, nous tentons de répondre au problème qui peut se poser dans ce domaine représenté dans la manière dont le législateur algérien traite le système de certification de conformité, partant du fait que le législateur est celui qui fixe le cadre juridique de toute politique poursuivie par l'État dans tous les domaines, y compris le domaine économique, et nous avons d'abord étudié les organismes d'évaluation de la conformité, puis les procédures de certification de conformité. Nous sommes arrivés à une conclusion générale qui est l'indifférence des institutions algériennes au système qualité et l'indifférence à celui-ci, surtout les petites et moyennes.
"""
result1 = separate_abstracts(text1)

# Example 2: Text with only two languages
text2 = """
English: This is an English abstract.
French: Ceci est un résumé français.
"""

result2 = separate_abstracts(text2)
result1, result2

({'english': 'The main objective of this study is to clarify the role played by the certification of conformity of products, or the institution obtaining a certificate of conformity for its products, in highlighting its economic position at national and international levels, and thus promoting the national economy, in particular because we know that Algeria entered the world market by opening the door to investments, as well as to the reforms it undertook in the late 1980s and early 1990s by adopting the market economy system and seeking membership in the World Trade Organization. Through this study, we try to respond to the problem that may arise in this field represented in the way in which the Algerian legislator treats the system of certification of conformity, starting from the fact that the legislator is the one who fixes the legal framework of any policy. pursued by the State in all fields, including the economic field, and we first studied conformity assessment bodies, then con

In [8]:
def is_latin_word(word):
    return all([c in 'abcdefghijklmnopqrstuvwxyz' for c in word])

def is_arabic_word(word):
    # Arabic Unicode range
    # Basic Arabic letters: 0600-06FF
    # Arabic Supplement: 0750-077F
    # Arabic Extended-A: 08A0-08FF
    # Arabic Presentation Forms-A: FB50-FDFF
    # Arabic Presentation Forms-B: FE70-FEFF
    
    if not isinstance(word, str):
        return False
    
    for char in word:
        if ('\u0600' <= char <= '\u06FF' or  # Basic Arabic
            '\u0750' <= char <= '\u077F' or   # Arabic Supplement
            '\u08A0' <= char <= '\u08FF' or   # Arabic Extended-A
            '\uFB50' <= char <= '\uFDFF' or   # Arabic Presentation Forms-A
            '\uFE70' <= char <= '\uFEFF'):    # Arabic Presentation Forms-B
            return True
    
    return False

def is_french_word(word):
    if not isinstance(word, str):
        return False
    
    # Define French-specific characters
    french_chars = set('éèêëàâäôöûüùïîçÉÈÊËÀÂÄÔÖÛÜÙÏÎÇ')
    # Basic Latin alphabet is also used in French    
    for char in word:
        if char in french_chars:
            return True
    
    return False

In [9]:
manually_excluded_articles= [
    'https://www.asjp.cerist.dz/en/article/34611'
]

In [10]:
english_arabic_paired_abstracts_counter = 0
arabic_abstracts_counter = 0
filtered_papers_with_metadata = {}
for paper_path_json in tqdm(papers_paths_json):# 
    try:
        paper_metadata = json.load(open(paper_path_json))
        if paper_metadata['url'] in manually_excluded_articles:
            continue
        if paper_metadata['abstract'] and paper_metadata['abstract'].strip():
            paper_metadata['abstract'] = ''.join(c for c in paper_metadata['abstract'] if c.isprintable()).strip()
            splitted_abstracts = separate_abstracts(paper_metadata['abstract'])
            if splitted_abstracts['arabic'] and not all(is_arabic_word(word) for word in splitted_abstracts['arabic'].split(' ')[-3:]):
                continue
            if splitted_abstracts['arabic'] and not all(is_arabic_word(word) for word in splitted_abstracts['arabic'].split(' ')[:2]):
                continue
            if splitted_abstracts['english'] and any(is_arabic_word(word) for word in splitted_abstracts['english'].split(' ')[:3]):
                continue
            if splitted_abstracts['english'] and any(is_arabic_word(word) for word in splitted_abstracts['english'].split(' ')[-3:]):
                continue
            if splitted_abstracts['french'] and any(is_arabic_word(word) for word in splitted_abstracts['french'].split(' ')[:3]):
                continue
            if splitted_abstracts['french'] and any(is_arabic_word(word) for word in splitted_abstracts['french'].split(' ')[-3:]):
                continue
            # from pprint import pprint
            # print(paper_metadata['abstract'])
            # pprint(splitted_abstracts)
            # print('-'*80)
            paper_metadata['english_abstract'] = splitted_abstracts['english']
            paper_metadata['arabic_abstract'] = splitted_abstracts['arabic']
            paper_metadata['french_abstract'] = splitted_abstracts['french']
            if paper_metadata['arabic_abstract']:
                arabic_abstracts_counter += 1
            if paper_metadata['english_abstract'] and paper_metadata['arabic_abstract']:
                english_arabic_paired_abstracts_counter += 1
            filtered_papers_with_metadata[paper_path_json] = paper_metadata
            with open(paper_path_json,'w') as f:
                json.dump(paper_metadata,f,ensure_ascii=False,indent=4)
    except Exception as e:
        print('error with paper', paper_path_json,e)

  0%|          | 0/133912 [00:00<?, ?it/s]

In [11]:
english_arabic_paired_abstracts_counter, arabic_abstracts_counter, len(filtered_papers_with_metadata)

(48707, 101020, 105666)

In [30]:
!mkdir -p tmpfiles
with open('tmpfiles/accepted_papers_with_metadata.json', 'w') as f:
    json.dump(filtered_papers_with_metadata, f, indent=4, ensure_ascii=False)

# Implement the code that gets text from the paper

In [31]:
from PyPDF2 import PdfReader

In [32]:
from difflib import SequenceMatcher

def remove_similar_first_last_lines_if_exists(pages_texts: dict, similarity_threshold: float = 0.75):
    # Split all pages into lines
    pages_lines = {
        page_number: page_text.splitlines()
        for page_number, page_text in pages_texts.items()
    }
    
    # Find the minimum number of lines across all pages
    min_lines = min(len(lines) for lines in pages_lines.values())
    
    # Keep track of how many lines to remove from start and end
    lines_to_remove_start = 0
    lines_to_remove_end = 0
    
    # Check lines from the start
    for line_idx in range(min_lines):
        # Get all lines at current position
        current_lines = [lines[line_idx].strip() for lines in pages_lines.values()]
        
        # Check if all lines are similar enough to the first page's line
        reference_line = current_lines[0]
        are_all_similar = all(
            SequenceMatcher(None, reference_line, line).ratio() >= similarity_threshold
            for line in current_lines[1:]
        )
        
        if are_all_similar:
            lines_to_remove_start = line_idx + 1
        else:
            break
    
    # Check lines from the end
    for line_idx in range(min_lines - lines_to_remove_start):
        # Get all lines at current position from the end
        current_lines = [lines[-(line_idx + 1)].strip() for lines in pages_lines.values()]
        
        # Check if all lines are similar enough to the first page's line
        reference_line = current_lines[0]
        are_all_similar = all(
            SequenceMatcher(None, reference_line, line).ratio() >= similarity_threshold
            for line in current_lines[1:]
        )
        
        if are_all_similar:
            lines_to_remove_end = line_idx + 1
        else:
            break
    
    # If we found similar lines at either end, remove them
    if lines_to_remove_start > 0 or lines_to_remove_end > 0:
        return {
            page_number: '\n'.join(lines[lines_to_remove_start:len(lines)-lines_to_remove_end or None]).strip()
            for page_number, lines in pages_lines.items()
        }
    
    return pages_texts

In [33]:
arabic_unicode_map = {
    'FE8D': '0627',  # ʾalif isolated
    'FE8E': '0627',  # ʾalif final
    'FE8F': '0628',  # bāʾ isolated
    'FE90': '0628',  # bāʾ final
    'FE92': '0628',  # bāʾ medial
    'FE91': '0628',  # bāʾ initial
    'FE95': '062A',  # tāʾ isolated
    'FE96': '062A',  # tāʾ final
    'FE98': '062A',  # tāʾ medial
    'FE97': '062A',  # tāʾ initial
    'FE99': '062B',  # ṯāʾ isolated
    'FE9A': '062B',  # ṯāʾ final
    'FE9C': '062B',  # ṯāʾ medial
    'FE9B': '062B',  # ṯāʾ initial
    'FE9D': '062C',  # ǧīm isolated
    'FE9E': '062C',  # ǧīm final
    'FEA0': '062C',  # ǧīm medial
    'FE9F': '062C',  # ǧīm initial
    'FEA1': '062D',  # ḥāʾ isolated
    'FEA2': '062D',  # ḥāʾ final
    'FEA4': '062D',  # ḥāʾ medial
    'FEA3': '062D',  # ḥāʾ initial
    'FEA5': '062E',  # ḫāʾ isolated
    'FEA6': '062E',  # ḫāʾ final
    'FEA8': '062E',  # ḫāʾ medial
    'FEA7': '062E',  # ḫāʾ initial
    'FEA9': '062F',  # dāl isolated
    'FEAA': '062F',  # dāl final
    'FEAB': '0630',  # ḏāl isolated
    'FEAC': '0630',  # ḏāl final
    'FEAD': '0631',  # rāʾ isolated
    'FEAE': '0631',  # rāʾ final
    'FEAF': '0632',  # zayn isolated
    'FEB0': '0632',  # zayn final
    'FEB1': '0633',  # sīn isolated
    'FEB2': '0633',  # sīn final
    'FEB4': '0633',  # sīn medial
    'FEB3': '0633',  # sīn initial
    'FEB5': '0634',  # šīn isolated
    'FEB6': '0634',  # šīn final
    'FEB8': '0634',  # šīn medial
    'FEB7': '0634',  # šīn initial
    'FEB9': '0635',  # ṣād isolated
    'FEBA': '0635',  # ṣād final
    'FEBC': '0635',  # ṣād medial
    'FEBB': '0635',  # ṣād initial
    'FEBD': '0636',  # ḍād isolated
    'FEBE': '0636',  # ḍād final
    'FEC0': '0636',  # ḍād medial
    'FEBF': '0636',  # ḍād initial
    'FEC1': '0637',  # ṭāʾ isolated
    'FEC2': '0637',  # ṭāʾ final
    'FEC4': '0637',  # ṭāʾ medial
    'FEC3': '0637',  # ṭāʾ initial
    'FEC5': '0638',  # ẓāʾ isolated
    'FEC6': '0638',  # ẓāʾ final
    'FEC8': '0638',  # ẓāʾ medial
    'FEC7': '0638',  # ẓāʾ initial
    'FEC9': '0639',  # ʿayn isolated
    'FECA': '0639',  # ʿayn final
    'FECC': '0639',  # ʿayn medial
    'FECB': '0639',  # ʿayn initial
    'FECD': '063A',  # ġayn isolated
    'FECE': '063A',  # ġayn final
    'FED0': '063A',  # ġayn medial
    'FECF': '063A',  # ġayn initial
    'FED1': '0641',  # fāʾ isolated
    'FED2': '0641',  # fāʾ final
    'FED4': '0641',  # fāʾ medial
    'FED3': '0641',  # fāʾ initial
    'FED5': '0642',  # qāf isolated
    'FED6': '0642',  # qāf final
    'FED8': '0642',  # qāf medial
    'FED7': '0642',  # qāf initial
    'FED9': '0643',  # kāf isolated
    'FEDA': '0643',  # kāf final
    'FEDC': '0643',  # kāf medial
    'FEDB': '0643',  # kāf initial
    'FEDD': '0644',  # lām isolated
    'FEDE': '0644',  # lām final
    'FEE0': '0644',  # lām medial
    'FEDF': '0644',  # lām initial
    'FEE1': '0645',  # mīm isolated
    'FEE2': '0645',  # mīm final
    'FEE4': '0645',  # mīm medial
    'FEE3': '0645',  # mīm initial
    'FEE5': '0646',  # nūn isolated
    'FEE6': '0646',  # nūn final
    'FEE8': '0646',  # nūn medial
    'FEE7': '0646',  # nūn initial
    'FEE9': '0647',  # hāʾ isolated
    'FEEA': '0647',  # hāʾ final
    'FEEC': '0647',  # hāʾ medial
    'FEEB': '0647',  # hāʾ initial
    'FEED': '0648',  # wāw isolated
    'FEEE': '0648',  # wāw final
    'FEF1': '064A',  # yāʾ isolated
    'FEF2': '064A',  # yāʾ final
    'FEF4': '064A',  # yāʾ medial
    'FEF3': '064A',  # yāʾ initial
    'FE81': '0622',  # ʾalif maddah isolated
    'FE82': '0622',  # ʾalif maddah final
    'FE93': '0629',  # tāʾ marbūṭah isolated
    'FE94': '0629',  # tāʾ marbūṭah final
    'FEEF': '0649',  # ʾalif maqṣūrah isolated
    'FEF0': '0649'   # ʾalif maqṣūrah final
}

def normalize_arabic_text(text):
    
    normalized_chars = []
    
    for char in text:
        # Get the unicode code point of the character in uppercase hexadecimal format
        char_code = format(ord(char), '04X')
        
        # If the character is in our mapping, convert it to the general form
        if char_code in arabic_unicode_map:
            # Convert the general unicode back to a character
            normalized_char = chr(int(arabic_unicode_map[char_code], 16))
            normalized_chars.append(normalized_char)
        else:
            # Keep characters that don't need normalization
            normalized_chars.append(char)
    
    # Join the characters back into a string
    return ''.join(normalized_chars)

In [34]:
import unicodedata
def clean_arabic_text(text):
    """
    Clean and normalize Arabic text extracted from PDF
    
    Args:
        text (str): Raw Arabic text from PDF
    Returns:
        str: Cleaned and normalized text
    """
    # Step 1: Normalize using NFKC form (best for Arabic)
    # This combines characters and their diacritics and standardizes presentation forms
    normalized = unicodedata.normalize('NFKC', text)
    
    # Step 2: Remove any remaining combining marks (optional)
    # Uncomment if you want to remove all diacritics
    # normalized = ''.join(c for c in normalized 
    #                     if not unicodedata.combining(c))
    
    # Step 3: Fix common PDF extraction issues
    normalized = (normalized
                 .replace('\u200f', '')  # Remove Right-to-Left Mark
                 .replace('\u200e', '')  # Remove Left-to-Right Mark
                 .replace('\u200b', '')  # Remove Zero Width Space
                 .replace('\u200c', '')  # Remove Zero Width Non-Joiner
                 .replace('\u200d', '')  # Remove Zero Width Joiner
                 .strip())
    
    # Step 4: Normalize spaces
    normalized = re.sub(r'\s+', ' ', normalized)
    normalized = normalize_arabic_text(normalized)
    return normalized

In [35]:
from langdetect import detect

def is_predominantly_arabic(pages_texts, 
                          page_threshold: float = 0.99,
                          line_threshold: float = 0.97):
    def is_arabic_line(line: str) -> bool:
        """Helper function to check if a line is Arabic."""
        if not line.strip():
            return False
        try:
            return detect(line) == 'ar'
        except:
            return False
    
    # Count how many pages are predominantly Arabic
    arabic_page_count = 0
    total_pages = len(pages_texts)
    
    # Check each page
    for page_text in pages_texts.values():
        # Split into lines and filter out empty lines
        lines = [line.strip() for line in page_text.splitlines() if line.strip()]
        if not lines:
            continue
            
        # Count Arabic lines
        arabic_lines = sum(1 for line in lines if is_arabic_line(line))
        total_lines = len(lines)
        
        # Check if this page is predominantly Arabic
        if total_lines > 0 and (arabic_lines / total_lines) >= line_threshold:
            arabic_page_count += 1
    
    # Check if enough pages are predominantly Arabic
    return (arabic_page_count / total_pages) >= page_threshold if total_pages > 0 else False

In [ ]:
# Suppress all warnings
warnings.filterwarnings("ignore")

# Create a dummy file to redirect output
null_file = open(os.devnull, "w")

extracted_papers_counter = 0

############ can be commented
# import random
# random.shuffle(filtered_papers_with_metadata)
###############################

papers_texts = {}

for paper_path_json in tqdm(filtered_papers_with_metadata.keys(), desc="Papers", position=0):
    # skip if paper text is extracted
    with open(paper_path_json, "r") as f:
        paper_metadata = json.load(f)
    if paper_metadata.get('paper_extracted_content'):
        continue
    paper_path_pdf = paper_path_json[::-1].replace("nosj.","fdp.", 1)[
        ::-1
    ]  # revert, replace, revert
    try:
        # Redirect both stdout and stderr while reading PDF
        with redirect_stdout(null_file), redirect_stderr(null_file):
            reader = PdfReader(paper_path_pdf)
            pages_texts = {}
            pages_are_images = False
            if len(reader.pages) > 50:
                # we want papers to be of length 50 maximum
                continue

            # Store the tqdm object in a variable
            pages_bar = tqdm(
                enumerate(reader.pages),
                total=len(reader.pages),
                desc="Pages",
                position=1,
                leave=False,
            )

            for j, page in pages_bar:
                start_time = time.time()
                page_text = page.extract_text()
                if page_text:
                    pages_texts[j] = "".join(c for c in page_text if c.isprintable())
                if time.time() - start_time > 1:
                    # if it is taking too much time, then pages are just images.
                    pages_are_images = True
                    pages_bar.n = pages_bar.total  # Set progress to 100%
                    pages_bar.refresh()  # Update display
                    pages_bar.close()  # Close the bar
                    break
        if not pages_are_images:
            # preprocess pages_texts
            # if the first line is the same across the pages, delete it, this usually happens with headers
            pages_texts = remove_similar_first_last_lines_if_exists(pages_texts)
            if not is_predominantly_arabic(pages_texts):
                continue
            pages_texts = {
                page_number: clean_arabic_text(page_text)
                for page_number, page_text in pages_texts.items()
            }
            # save the extracted paper text
            pages_texts_str = "\n".join(pages_texts.values())
            paper_metadata["paper_extracted_content"] = pages_texts_str
            with open(paper_path_json, "w") as f:
                # print(extracted_papers_counter,paper_path_json)
                json.dump(paper_metadata, f, ensure_ascii=False, indent=4)
                # if extracted_papers_counter % 100 == 0:
                #     print('extracted papers count:',extracted_papers_counter)

            papers_texts[paper_path_pdf] = pages_texts
        extracted_papers_counter += 1
        # if extracted_papers_counter >= 100:
        #     break

    except Exception as e:
        print("error occured with paper:", paper_path_pdf, ". The error is: ", e)
        continue

# Don't forget to close the null file
null_file.close()

Papers:   0%|          | 0/58768 [00:00<?, ?it/s]

Pages:   0%|          | 0/30 [00:00<?, ?it/s]

Pages:   0%|          | 0/16 [00:00<?, ?it/s]

Pages:   0%|          | 0/31 [00:00<?, ?it/s]

Pages:   0%|          | 0/23 [00:00<?, ?it/s]

...

Pages:   0%|          | 0/22 [00:00<?, ?it/s]

Pages:   0%|          | 0/13 [00:00<?, ?it/s]

Pages:   0%|          | 0/12 [00:00<?, ?it/s]

Pages:   0%|          | 0/24 [00:00<?, ?it/s]

Pages:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
# !mkdir -p tmpfiles/papers_texts
# for i,(paper_path_pdf, pages_texts) in enumerate(tqdm(papers_texts.items(), desc='Papers', position=0)):
#     # write them to tmpfiles/papers_texts folder
#     with open(f'tmpfiles/papers_texts/{i}.json', 'w') as f:
#         f.write(json.dumps(pages_texts, ensure_ascii=False, indent=4))

In [ ]:
# for i,(paper_path_pdf, pages_texts) in enumerate(tqdm(papers_texts.items(), desc='Papers', position=0)):
#     paper_path_json = paper_path_pdf[::-1].replace('fdp.','nosj.',1)[::-1] # revert, replace, revert
#     pages_texts_str = '\n'.join(pages_texts.values())
#     try:
#         with open(paper_path_json,'r') as f:
#             paper_metadata = json.load(f)
#         paper_metadata['paper_extracted_content'] = pages_texts_str
#         with open(paper_path_json,'w') as f:
#             json.dump(paper_metadata,f,ensure_ascii=False, indent=4)
#     except Exception as e:
#         print("Error with paper:", paper_path_json, e)